In [1]:
import os
import pandas as pd
import numpy as np
import torch

from utils.data import load_training_test_data

Check GPU support

In [2]:
torch.cuda.is_available()

True

Load data

In [3]:
(TRAIN_DATAFRAME, TEST_DATAFRAME) = \
  load_training_test_data(os.path.join('data', 'train.json'),
                          os.path.join('data', 'test.json'))

In [4]:
list(TRAIN_DATAFRAME.columns)

['id',
 'bathrooms',
 'bedrooms',
 'building_id',
 'created',
 'description',
 'display_address',
 'features',
 'latitude',
 'listing_id',
 'longitude',
 'manager_id',
 'photos',
 'price',
 'street_address',
 'interest_level']